In [1]:
"""This script will use esm models to generate embeddings for PPIs"""
import torch
import esm
import pandas as pd
import scanpy as sc
import numpy as np
from fuzzywuzzy import fuzz

In [5]:
df = pd.read_csv('./data/high_confidence_ppi.csv', index_col=0)
df

,Unnamed: 0,UniProtID_1,UniProtID_2,symbol_1,symbol_2,seq_1,seq_2,Source,Organism Interactor A,Organism Interactor B,...,Tags,Cell line specificity,Read count,Odds ratio,BH-corrected p-value,Potential background contamination,len_1,len_2,complex_len,unique_key
0,0,P67809,Q13242,YBOX1_HUMAN,SRSF9_HUMAN,MSSEAETQQPPAAPPAAPALSAADTKPGTTGSGAGSGGPGGLTSAA...,MSGWADERGGEGDGRIYVGNLPTDVREKDLEDLFYKYGRIREIELK...,bioGRID,9606.0,9606.0,...,-,NaN,NaN,NaN,NaN,NaN,324,221,545,P67809#Q13242
1,1,Q8NER1,Q86SS6,TRPV1_HUMAN,SYT9_HUMAN,MKKWSSTDLGAAADPLQKDTCPDPLDGDPNSRPPPAKPQLSTAKSR...,MPGARDALCHQALQLLAELCARGALEHDSCQDFIYHLRDRARPRLR...,bioGRID,9606.0,9606.0,...,-,NaN,NaN,NaN,NaN,NaN,839,491,1330,Q86SS6#Q8NER1
2,2,O75319,Q13242,DUS11_HUMAN,SRSF9_HUMAN,MRNSETLERGVGGCRVFSCLGSYPGIEGAGLALLADLALGGRLLGT...,MSGWADERGGEGDGRIYVGNLPTDVREKDLEDLFYKYGRIREIELK...,bioGRID,9606.0,9606.0,...,-,NaN,NaN,NaN,NaN,NaN,377,221,598,O75319#Q13242
3,3,Q13526,Q7Z5J4,PIN1_HUMAN,RAI1_HUMAN,MADEEKLPPGWEKRMSRSSGRVYYFNHITNASQWERPSGNSSSGGK...,MQSFRERCGFHGKQQNYQQTSQETSRLENYRQPSQAGLSCDRQRLL...,bioGRID,9606.0,9606.0,...,-,NaN,NaN,NaN,NaN,NaN,163,1906,2069,Q13526#Q7Z5J4
4,4,Q13526,Q9UJY4,PIN1_HUMAN,GGA2_HUMAN,MADEEKLPPGWEKRMSRSSGRVYYFNHITNASQWERPSGNSSSGGK...,MAATAVAAAVAGTESAQGPPGPAASLELWLNKATDPSMSEQDWSAI...,bioGRID,9606.0,9606.0,...,-,NaN,NaN,NaN,NaN,NaN,163,613,776,Q13526#Q9UJY4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24516,24516,P60763,Q13443,RAC3_HUMAN,ADAM9_HUMAN,MQAIKCVVVGDGAVGKTCLLISYTTNAFPGEYIPTVFDNYSANVMV...,MGSGARFPSGTLRVRWLLLLGLVGPVLGAARPGFQQTSHLSSYEII...,bioGRID,9606.0,9606.0,...,-,NaN,NaN,NaN,NaN,NaN,192,819,1011,P60763#Q13443
24517,24517,P60763,Q9UNK0,RAC3_HUMAN,STX8_HUMAN,MQAIKCVVVGDGAVGKTCLLISYTTNAFPGEYIPTVFDNYSANVMV...,MAPDPWFSTYDSTCQIAQEIAEKIQQRNQYERKGEKAPKLTVTIRA...,bioGRID,9606.0,9606.0,...,-,NaN,NaN,NaN,NaN,NaN,192,236,428,P60763#Q9UNK0
24518,24518,Q9BQ90,Q15843,KLDC3_HUMAN,NEDD8_HUMAN,MLRWTVHLEGGPRRVNHAAVAVGHRVYSFGGYCSGEDYETLRQIDV...,MLIKVKTLTGKEIEIDIEPTDKVERIKERVEEKEGIPPQQQRLIYS...,bioGRID,9606.0,9606.0,...,-,NaN,NaN,NaN,NaN,NaN,382,81,463,Q15843#Q9BQ90
24519,24519,Q9BWK5,Q9H9Q4,CYREN_HUMAN,NHEJ1_HUMAN,METLQSETKTRVLPSWLTAQVATKNVAPMKAPKRMRMAAVPVAAAR...,MEELEQGLLMQPWAWLQLAENSLLAKVFITKQGYALLVSDLQQVWH...,bioGRID,9606.0,9606.0,...,-,NaN,NaN,NaN,NaN,NaN,157,299,456,Q9BWK5#Q9H9Q4


In [8]:
columns_to_keep = ['UniProtID_1', 'UniProtID_2', 'symbol_1', 'symbol_2', 'seq_1', 'seq_2',
                   'Experimental System', 'Throughput', 'len_1', 'len_2', 'Pubmed ID']
df = df[columns_to_keep]
df.drop_duplicates(inplace=True) # no duplicate
df

,UniProtID_1,UniProtID_2,symbol_1,symbol_2,seq_1,seq_2,Experimental System,Throughput,len_1,len_2,Pubmed ID
0,P67809,Q13242,YBOX1_HUMAN,SRSF9_HUMAN,MSSEAETQQPPAAPPAAPALSAADTKPGTTGSGAGSGGPGGLTSAA...,MSGWADERGGEGDGRIYVGNLPTDVREKDLEDLFYKYGRIREIELK...,Two-hybrid,Low Throughput,324,221,12604611.0
1,Q8NER1,Q86SS6,TRPV1_HUMAN,SYT9_HUMAN,MKKWSSTDLGAAADPLQKDTCPDPLDGDPNSRPPPAKPQLSTAKSR...,MPGARDALCHQALQLLAELCARGALEHDSCQDFIYHLRDRARPRLR...,Two-hybrid,Low Throughput,839,491,15066994.0
2,O75319,Q13242,DUS11_HUMAN,SRSF9_HUMAN,MRNSETLERGVGGCRVFSCLGSYPGIEGAGLALLADLALGGRLLGT...,MSGWADERGGEGDGRIYVGNLPTDVREKDLEDLFYKYGRIREIELK...,Two-hybrid,Low Throughput,377,221,9685386.0
3,Q13526,Q7Z5J4,PIN1_HUMAN,RAI1_HUMAN,MADEEKLPPGWEKRMSRSSGRVYYFNHITNASQWERPSGNSSSGGK...,MQSFRERCGFHGKQQNYQQTSQETSRLENYRQPSQAGLSCDRQRLL...,Two-hybrid,High Throughput,163,1906,16189514.0
4,Q13526,Q9UJY4,PIN1_HUMAN,GGA2_HUMAN,MADEEKLPPGWEKRMSRSSGRVYYFNHITNASQWERPSGNSSSGGK...,MAATAVAAAVAGTESAQGPPGPAASLELWLNKATDPSMSEQDWSAI...,Two-hybrid,High Throughput,163,613,16189514.0
...,...,...,...,...,...,...,...,...,...,...,...
24516,P60763,Q13443,RAC3_HUMAN,ADAM9_HUMAN,MQAIKCVVVGDGAVGKTCLLISYTTNAFPGEYIPTVFDNYSANVMV...,MGSGARFPSGTLRVRWLLLLGLVGPVLGAARPGFQQTSHLSSYEII...,Affinity Capture-MS,Low Throughput,192,819,31871319.0
24517,P60763,Q9UNK0,RAC3_HUMAN,STX8_HUMAN,MQAIKCVVVGDGAVGKTCLLISYTTNAFPGEYIPTVFDNYSANVMV...,MAPDPWFSTYDSTCQIAQEIAEKIQQRNQYERKGEKAPKLTVTIRA...,Affinity Capture-MS,Low Throughput,192,236,31871319.0
24518,Q9BQ90,Q15843,KLDC3_HUMAN,NEDD8_HUMAN,MLRWTVHLEGGPRRVNHAAVAVGHRVYSFGGYCSGEDYETLRQIDV...,MLIKVKTLTGKEIEIDIEPTDKVERIKERVEEKEGIPPQQQRLIYS...,Affinity Capture-MS,Low Throughput,382,81,35468939.0
24519,Q9BWK5,Q9H9Q4,CYREN_HUMAN,NHEJ1_HUMAN,METLQSETKTRVLPSWLTAQVATKNVAPMKAPKRMRMAAVPVAAAR...,MEELEQGLLMQPWAWLQLAENSLLAKVFITKQGYALLVSDLQQVWH...,Affinity Capture-Western,Low Throughput,157,299,30017584.0


In [10]:
# generate embeddings
seqs_wt1 = df.seq_1.values.tolist()
seqs_wt2 = df.seq_2.values.tolist()
seqs_wt1 = set(seqs_wt1)
seqs_wt2 = set(seqs_wt2)
seqs_labeled_wt1 = []
count = 0
for seq in seqs_wt1:
    seqs_labeled_wt1.append(tuple((str('seq' + str(count)), seq)))
    count += 1
seqs_labeled_wt2 = []
count = 0
for seq in seqs_wt2:
    seqs_labeled_wt2.append(tuple((str('seq' + str(count)), seq)))
    count += 1

In [11]:
# Load ESM-2 model
torch.cuda.empty_cache()
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()  # disables dropout for deterministic results

ESM2(
  (embed_tokens): Embedding(33, 1280, padding_idx=1)
  (layers): ModuleList(
    (0): TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=1280, out_features=5120, bias=True)
      (fc2): Linear(in_features=5120, out_features=1280, bias=True)
      (final_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    )
    (1): TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bia

In [12]:
# find longest sequence
len(max(list(zip(*seqs_labeled_wt1))[1], key=len))

5762

In [13]:
# alternative way to generate batches
from torch.utils.data import TensorDataset
from esm import Alphabet, FastaBatchedDataset
batch_size = 1000
dataset = FastaBatchedDataset(list(zip(*seqs_labeled_wt1))[0], list(zip(*seqs_labeled_wt1))[1])
batches = dataset.get_batch_indices(batch_size, extra_toks_per_seq=1)
data_loader = torch.utils.data.DataLoader(dataset, collate_fn=Alphabet.from_architecture("roberta_large").get_batch_converter(),
            batch_sampler=batches, pin_memory=True)
dataset_seq2 = FastaBatchedDataset(list(zip(*seqs_labeled_wt2))[0], list(zip(*seqs_labeled_wt2))[1])
batches_seq2 = dataset_seq2.get_batch_indices(batch_size, extra_toks_per_seq=1)
data_loader_seq2 = torch.utils.data.DataLoader(dataset_seq2, collate_fn=Alphabet.from_architecture("roberta_large").get_batch_converter(), batch_sampler=batches_seq2, pin_memory=True)

In [14]:
if torch.cuda.is_available():
    model = model.cuda()
    print('Transferred model to GPU')

Transferred model to GPU


In [15]:
# QC
for batch_idx, (labels, strs, toks) in enumerate(data_loader):
    try:
        print(batch_idx,labels)
    except KeyError:
        print(batch_idx)

0 ['seq400', 'seq2159', 'seq9', 'seq1752', 'seq1063', 'seq3285', 'seq3627', 'seq3204', 'seq2911', 'seq101', 'seq1191', 'seq3447', 'seq471', 'seq3423']
1 ['seq2445', 'seq1104', 'seq958', 'seq2643', 'seq49', 'seq291', 'seq3097', 'seq1393', 'seq2836', 'seq1693', 'seq2298', 'seq960']
2 ['seq1542', 'seq1685', 'seq1779', 'seq477', 'seq531', 'seq3253', 'seq35', 'seq681', 'seq1801', 'seq1961', 'seq3386']
3 ['seq3486', 'seq3041', 'seq1763', 'seq2824', 'seq395', 'seq654', 'seq1363', 'seq3510', 'seq391', 'seq3066']
4 ['seq203', 'seq2192', 'seq3225', 'seq372', 'seq969', 'seq1519', 'seq3014', 'seq27', 'seq2858', 'seq1807']
5 ['seq2936', 'seq3142', 'seq3470', 'seq60', 'seq1640', 'seq1694', 'seq496', 'seq1048', 'seq1872', 'seq2403']
6 ['seq563', 'seq3681', 'seq251', 'seq2334', 'seq2375', 'seq2900', 'seq3624', 'seq309', 'seq370']
7 ['seq536', 'seq547', 'seq883', 'seq3293', 'seq474', 'seq2226', 'seq1139', 'seq880', 'seq451']
8 ['seq1791', 'seq2995', 'seq556', 'seq3155', 'seq3509', 'seq8', 'seq1420', 's

In [16]:
representation_store_dict = {}
for batch_idx, (labels, strs, toks) in enumerate(data_loader):
    if torch.cuda.is_available():
        toks = toks.to(device='cuda', non_blocking=True)
    with torch.no_grad():
        results = model(toks, repr_layers = [33], return_contacts = True)['representations'][33]
    results_cpu = results.to(device='cpu')
    for i, str_ in enumerate(strs):
        # only select representations relate to the sequence
        # rest of the sequences are paddings, check notebook
        # create dictionary {sequence: embeddings}
        representation_store_dict[str_] = results_cpu[i, 1: (len(strs[i])+1)].numpy()

torch.Size([14, 70, 1280])
torch.Size([12, 83, 1280])
torch.Size([11, 88, 1280])
torch.Size([10, 95, 1280])
torch.Size([10, 99, 1280])
torch.Size([10, 101, 1280])
torch.Size([9, 105, 1280])
torch.Size([9, 110, 1280])
torch.Size([8, 113, 1280])
torch.Size([8, 115, 1280])
torch.Size([8, 118, 1280])
torch.Size([8, 121, 1280])
torch.Size([8, 126, 1280])
torch.Size([7, 127, 1280])
torch.Size([7, 128, 1280])
torch.Size([7, 130, 1280])
torch.Size([7, 132, 1280])
torch.Size([7, 133, 1280])
torch.Size([7, 136, 1280])
torch.Size([7, 138, 1280])
torch.Size([7, 138, 1280])
torch.Size([7, 140, 1280])
torch.Size([7, 142, 1280])
torch.Size([7, 143, 1280])
torch.Size([6, 144, 1280])
torch.Size([6, 146, 1280])
torch.Size([6, 147, 1280])
torch.Size([6, 148, 1280])
torch.Size([6, 150, 1280])
torch.Size([6, 152, 1280])
torch.Size([6, 153, 1280])
torch.Size([6, 154, 1280])
torch.Size([6, 155, 1280])
torch.Size([6, 156, 1280])
torch.Size([6, 157, 1280])
torch.Size([6, 159, 1280])
torch.Size([6, 160, 1280])


RuntimeError: CUDA out of memory. Tried to allocate 5.06 GiB (GPU 0; 39.59 GiB total capacity; 27.90 GiB already allocated; 4.99 GiB free; 32.56 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# take the average of the representations of the proteins
# for umaps, along axis 0
sequence_embeddings = {key: np.mean(value, axis=0, keepdims=True) for key, value in representation_store_dict.items()}

In [ ]:
representation_store_dict_seq2 = {}
for batch_idx, (labels, strs, toks) in enumerate(data_loader_seq2):
    if torch.cuda.is_available():
        toks = toks.to(device='cuda', non_blocking=True)
    with torch.no_grad():
        results = model(toks, repr_layers = [33], return_contacts = True)['representations'][33]
        #results = model(toks, repr_layers = [33], return_contacts = True)['logits']
    results_cpu = results.to(device='cpu')
    for i, str_ in enumerate(strs):
        representation_store_dict_seq2[str_] = results_cpu[i, 1: (len(strs[i])+1)].numpy()
sequence_embeddings_seq2 = {key: np.mean(value, axis=0, keepdims=True) for key, value in representation_store_dict_seq2.items()}

In [ ]:
def update_embeddings(row, embedding_dict):
    """
    add embeddings to the metadata column.
    cannot do the reverse, because due to mislabel, several different protein names share the same sequences
    but as long as sequences are correct, so will the embeddings
    """
    for key, value in embedding_dict.items():
        if row == key:
            return value
df['wild_seq_1_embeddings'] = df['wild_seq_1'].apply(update_embeddings, embedding_dict=sequence_embeddings)
df['wild_seq_2_embeddings'] = df['wild_seq_2'].apply(update_embeddings, embedding_dict=sequence_embeddings_seq2)

In [ ]:
df.to_hdf('./outputs/dataframes/PPIs_embeddings_meta.hdf', key='df', mode='w')

In [ ]:
import pickle
import os, sys
path = './outputs/variables/'
if not os.path.exists(path):
    os.makedirs(path)
with open('PPI_seq1_embeddings_full.pk', 'wb') as f:
    pickle.dump(representation_store_dict, f)
with open('PPI_seq2_embeddings_full.pk', 'wb') as f2:
    pickle.dump(representation_store_dict_seq2, f2)